In [36]:
import numpy as np
import os
import matplotlib.pyplot as plt
import mcg_gro
import mdtraj as md

In [37]:
frames2us =  2e-9

# VERSION 1
## Reading directly from the .xtc file using mdtraj

In [38]:
# Input files
folder = "../data/T_250_P_500/simulation_co2_system_N_2000k_222000k_T_250_P_500_v3"

xtc_file = os.path.join(folder, "output.xtc")
topology_file = os.path.join(folder, "conf.gro")

# Load the trajectory with a stride of n (e.g., every 10th frame)
n = 50  # Change this to the desired interval
traj = md.load(xtc_file, top=topology_file, stride=n)
original_frame_indices = np.arange(traj.n_frames) * n

# Save the reduced trajectory as a .gro file
# Output file
# gro_file = "trajectory.gro"
# traj.save_gro(gro_file)

print(f"Trajectory loaded using every {n}th step")


KeyboardInterrupt: 

In [ ]:
folder_path = "../data/T_250_P_500/simulation_co2_system_N_2000k_222000k_T_250_P_500_v3/"
outfile_path = os.path.join(folder_path, f"mcg_results_skip_{n}")

times_ps, CAR_COM_frames, WATER_COM_frames, box_lengths = mcg_gro.COM_calculation_mdtraj(traj)

MCG_OP_results = []

with open(outfile_path, 'w') as outFile:
    # Write the header
    outFile.write(f"{'OG frame idx':<15}{'Frame':<10}{'Time (ps)':<20}{'MCG_OP':<15}\n")

    for idx, time_ps in enumerate(times_ps):
        CAR_COM = CAR_COM_frames[idx]
        WATER_COM = WATER_COM_frames[idx]
        box_length = box_lengths[idx]
        time_us = time_ps * 1e-6
        
        frame = round(time_us / frames2us)
        
        # Assuming `MCG` takes these arguments and computes the order parameter
        MCG_OP, _, monomer_coords = mcg_gro.MCG(CAR_COM, WATER_COM, box_length, order=3, guest_cutoff=0.9)
        #np.savetxt( '../mcg_tests/monomers.xyz', monomer_coords)
        print(f'The largest cluster size in the system at frame {frame} and time {round(time_ps, 10)} is: {MCG_OP}')
        MCG_OP_results.append(MCG_OP)
        outFile.write(f"{original_frame_indices[idx]:<15}{frame:<10}{round(time_ps, 10):<20}{MCG_OP:<15.6f}\n")

The largest cluster size in the system at frame 2005000 and time 4010.0 is: 7
The largest cluster size in the system at frame 2255000 and time 4510.0 is: 6
The largest cluster size in the system at frame 2505000 and time 5010.0 is: 10
The largest cluster size in the system at frame 2755000 and time 5510.0 is: 15
The largest cluster size in the system at frame 3005000 and time 6010.0 is: 10
The largest cluster size in the system at frame 3255000 and time 6510.0 is: 8
The largest cluster size in the system at frame 3505000 and time 7010.0 is: 9
The largest cluster size in the system at frame 3755000 and time 7510.00048828125 is: 13
The largest cluster size in the system at frame 4005000 and time 8010.0 is: 9
The largest cluster size in the system at frame 4255000 and time 8510.0 is: 10
The largest cluster size in the system at frame 4505000 and time 9010.0 is: 8
The largest cluster size in the system at frame 4755000 and time 9510.0 is: 8
The largest cluster size in the system at frame 5

In [ ]:
shooting_point = md.load_frame(xtc_file, index=original_frame_indices[2], top=topology_file)

In [ ]:
print(shooting_point.time)

[5010.]


# VERSION 2a

## Reading specific .gro input file 
(can be converted using gromacs (ex. _gmx trjconv -f output.xtc -s ../../../systems/co2_system/conf.gro -o trajectory_skip_5000.gro -skip 5000)_)

In [ ]:
folder_path = "../data/T_250_P_500/simulation_co2_system_N_2000k_222000k_T_250_P_500_v3/"
outfile_path = os.path.join(folder_path, "mcg_results")
infile_path = os.path.join(folder_path, "trajectory_skip_5000.gro")

frames, CAR_COM_frames, WATER_COM_frames, box_lengths = mcg_gro.COM_calculation_gro_multiframe(infile_path)
# frames, CAR_COM_frames, WATER_COM_frames, box_lengths = mcg_gro.COM_calculation_mdtraj(traj)

MCG_OP_results = []

with open(outfile_path, 'w') as outFile:
    # Write the header
    outFile.write(f"{'Frame':<10}{'Time (ps)':<20}{'MCG_OP':<15}\n")

    for idx, frame in enumerate(frames):
        CAR_COM = CAR_COM_frames[idx]
        WATER_COM = WATER_COM_frames[idx]
        box_length = box_lengths[idx]

        time = frame * frames2us
        print(time)
        
        # Assuming `MCG` takes these arguments and computes the order parameter
        MCG_OP, _, monomer_coords = mcg_gro.MCG(CAR_COM, WATER_COM, box_length, order=3, guest_cutoff=0.9)
        #np.savetxt( '../mcg_tests/monomers.xyz', monomer_coords)
        print(f'The largest cluster size in the system at frame {frame} is: {MCG_OP}')
        MCG_OP_results.append(MCG_OP)
        outFile.write(f"{frame:<10}{time:<20}{MCG_OP:<15.6f}\n")

0.0040100000000000005
The largest cluster size in the system at frame 2005000 is: 7
0.05401
The largest cluster size in the system at frame 27005000 is: 16
0.10401
The largest cluster size in the system at frame 52005000 is: 79
0.15401
The largest cluster size in the system at frame 77005000 is: 129
0.20401000000000002
The largest cluster size in the system at frame 102005000 is: 229
0.25401
The largest cluster size in the system at frame 127005000 is: 353
0.30401
The largest cluster size in the system at frame 152005000 is: 416
0.35401000000000005
The largest cluster size in the system at frame 177005000 is: 459
0.40401000000000004
The largest cluster size in the system at frame 202005000 is: 476


# VERSION 2a

## Reading all .gro input files of same name in given directory
(can be converted using gromacs (ex. _gmx trjconv -f output.xtc -s ../../../systems/co2_system/conf.gro -o trajectory_skip_5000.gro -skip 5000)_)

In [ ]:
infile_name = 'trajectory_skip_1000.gro'
outfile_name = 'mcg_results.txt'

T = 250
system_path = f'../data/T_{T}_P_500/'

for folder_name in os.listdir(system_path):
    folder_path = os.path.join(system_path, folder_name)
    if os.path.isdir(folder_path):
        infile_path = os.path.join(folder_path, infile_name)
        if os.path.exists(infile_path) and os.path.getsize(infile_path) != 0:

            outfile_path = os.path.join(folder_path, outfile_name)

            frames, CAR_COM_frames, WATER_COM_frames, box_lengths = mcg_gro.COM_calculation_gro_multiframe(infile_path)

            MCG_OP_results = []

            with open(outfile_path, 'w') as outFile:
                # Write the header
                outFile.write(f"{'Frame':<10}{'Time (ps)':<20}{'MCG_OP':<15}\n")

                for idx, frame in enumerate(frames):
                    CAR_COM = CAR_COM_frames[idx]
                    WATER_COM = WATER_COM_frames[idx]
                    box_length = box_lengths[idx]

                    time = frame * frames2us
                    
                    # Assuming `MCG` takes these arguments and computes the order parameter
                    MCG_OP, _, monomer_coords = mcg_gro.MCG(CAR_COM, WATER_COM, box_length, order=3, guest_cutoff=0.9)
                    #np.savetxt( '../mcg_tests/monomers.xyz', monomer_coords)
                    print(f'The largest cluster size in the system at frame {frame} is: {MCG_OP}')
                    MCG_OP_results.append(MCG_OP)
                    outFile.write(f"{frame:<10}{time:<20}{MCG_OP:<15.6f}\n")

The largest cluster size in the system at frame 222005000 is: 473
The largest cluster size in the system at frame 227005000 is: 472
The largest cluster size in the system at frame 232005000 is: 484
The largest cluster size in the system at frame 237005000 is: 488
The largest cluster size in the system at frame 242005000 is: 480
The largest cluster size in the system at frame 247005000 is: 474
The largest cluster size in the system at frame 252005000 is: 487
The largest cluster size in the system at frame 257005000 is: 491
The largest cluster size in the system at frame 262005000 is: 481
The largest cluster size in the system at frame 267005000 is: 487
The largest cluster size in the system at frame 272005000 is: 494
The largest cluster size in the system at frame 277005000 is: 493
The largest cluster size in the system at frame 282005000 is: 496
The largest cluster size in the system at frame 287005000 is: 491
The largest cluster size in the system at frame 292005000 is: 492
The larges

KeyboardInterrupt: 